In [9]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import time
from datetime import datetime
from decouple import config
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [15]:
def scrape_links_news(yesterday,date, keywords,page_number):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://banten.antaranews.com/search?q={keywords}&startDate={yesterday}&endDate={date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})

    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    print(f"data{page_number} and url {url}")
    # logger.success({
    #     "message": f"Scraped {len(page_links)} links from page {page_number} when {date}"
    #     })
    
    return page_links

In [16]:
def scrape_link_per_day(date, keywords):
    page_number=1
    page_links = None
    date = dt.datetime.strptime(date, "%Y-%m-%d")
    formatted_date = date.strftime("%d-%m-%Y")
    
    yesterday = date - dt.timedelta(days=1)
    yesterday = yesterday.strftime("%d-%m-%Y")
    
    page_links= scrape_links_news(yesterday,formatted_date, keywords,page_number)

    return page_links

In [17]:
link=scrape_link_per_day("2023-11-21",'jokowi')

data1 and url https://banten.antaranews.com/search?q=jokowi&startDate=20-11-2023&endDate=21-11-2023&page=1


In [20]:
link

['https://banten.antaranews.com/berita/265032/presiden-jokowi-tegaskan-indonesia-perjuangan-kemerdekaan-palestina',
 'https://banten.antaranews.com/berita/265026/presiden-jokowi-lepas-pengiriman-bantuan-tahap-kedua-untuk-palestina']

In [18]:
print(len(link))

2


In [19]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # element Title 
                    title_elem = soup.find('h1',{"class": "post-title"})
                    title_text = title_elem.text.strip() if title_elem else "Title not found"
                    # element Date
                    date_elem = soup.find('span', {"class": "article-date"})
                    date_text = date_elem.text.strip() if date_elem else "Date not found"
                    # locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')
                    month_names_id = {
                        'Januari': 'January',
                        'Februari': 'February',
                        'Maret': 'March',
                        'April': 'April',
                        'Mei': 'May',
                        'Juni': 'June',
                        'Juli': 'July',
                        'Agustus': 'August',
                        'September': 'September',
                        'Oktober': 'October',
                        'November': 'November',
                        'Desember': 'December',
                    }
                    for ind, eng in month_names_id.items():
                            date_text = date_text.replace(ind, eng)
                    day_names_id = {
                                    'Senin': 'Monday',
                                    'Selasa': 'Tuesday',
                                    'Rabu': 'Wednesday',
                                    'Kamis': 'Thursday',
                                    'Jumat': 'Friday',
                                    'Sabtu': 'Saturday',
                                    'Minggu': 'Sunday'
                                }
                    for ind, eng in day_names_id.items():
                                date_text = date_text.replace(ind, eng)
                    # locale.setlocale(locale.LC_TIME, 'id_ID')
                    date_obj = datetime.strptime(date_text, '%A, %d %B %Y %H:%M %Z')
                    formatted_date = date_obj.strftime('%Y-%m-%d')

                    body_elem = soup.find('div', {"class": "post-content clearfix"})
                    if body_elem:
                        content_text = body_elem.text
                        content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                        content_text = ' '.join(content_text.split())
                    else:
                        content_text ="Content not found"

                    return {
                        'title': title_text,
                        'date': formatted_date,
                        'content': content_text,
                        'link': url
                    }
                else:
                    # logger.error(
                    #     {"message": f"Failed to retrieve data from {url}: Status Code {response.status_code}"}
                    #     )
                    print('hellow')
            except requests.exceptions.RequestException as e:
                # logger.error({
                #     "message":f"Error fetching URL '{url}': {e}"
                # })
                print(e)
            except Exception as e:
                # logger.error({
                #     "message":f"Error processing URL '{url}': {e}"
                # })
                print(e)
            retries += 1
            if retries < max_retries:
                # logger.info({
                #     "message":f"Retrying {url} (Attempt {retries}/{max_retries})"
                # })
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None       


In [21]:
url='https://banten.antaranews.com/berita/265032/presiden-jokowi-tegaskan-indonesia-perjuangan-kemerdekaan-palestina'
cek=scrape_url(url)

In [22]:
print(cek)

{'title': 'Presiden Jokowi tegaskan Indonesia perjuangan kemerdekaan Palestina', 'date': '2023-11-20', 'content': 'Jakarta (ANTARA) - Presiden RI Joko Widodo (Jokowi) menegaskan bahwa Indonesia akan terus mendukung perjuangan kemerdekaan bangsa Palestina di tengah agresi Israel."Sekali lagi saya tegaskan Indonesia akan terus bersama mendukung perjuangan bangsa Palestina," kata Presiden ketika melepas pengiriman bantuan di Base Ops Pangkalan Udara (Lanud) TNI AU Halim Perdanakusuma, Jakarta, Senin.Tidak hanya bantuan kemanusiaan, kata Jokowi, Indonesia juga berupaya memberikan dukungan politik dengan menggalang dukungan dari negara-negara lain.Indonesia pun telah ditunjuk menjadi salah satu utusan Organisasi Kerja Sama Islam (OKI) untuk perdamaian Palestina berdasarkan KTT Luar Biasa OKI di Riyadh, Arab Saudi, 11 November lalu.Baca juga: Presiden Jokowi lepas pengiriman bantuan tahap kedua untuk PalestinaPresiden Jokowi langsung menindaklanjuti mandat tersebut dengan mendesak Presiden A